# [CoE202] **[Homework3]** CNN Classification for CIFAR10 (Pytorch)

## **Testing Script**

In this section, you are going to **test** CNN classification for CIFAR10 in the Pytorch framework.


## 0. Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt # this is for importing matplotlib.pyplot (library for graph plot)
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##1. Define your model to load weights, and define your transfomation in Dataloader

In [2]:
#You can only change this cell

#define your model
class CIFAR_Net(nn.Module): 
    def __init__(self):
        super(CIFAR_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.conv1 = nn.Conv2d(3, 80, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(80, 160, kernel_size=2, stride=1)
        self.conv3 = nn.Conv2d(160, 320, kernel_size=2, stride=1)
        self.conv4 = nn.Conv2d(320, 640, kernel_size=3, stride=1)

        self.pool = nn.MaxPool2d(kernel_size=2)

        self.fc1 = nn.Linear(640, 80)
        self.fc2 = nn.Linear(80, 10)

    def forward(self, x):
        x = F.relu(self.pool(self.conv1(x)))
        x = F.relu(self.pool(self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 640)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

#define your transformation on input image, groundtruth
transform = transforms.Compose([
    transforms.ToTensor(),
])
target_transform = None


## 2. Load test dataset

In [3]:
test_dataset = torchvision.datasets.CIFAR10("../cifar10_data/",
                                              download=True,
                                              train=False,
                                              transform=transform,
                                              target_transform=target_transform)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle=False, drop_last = True)




Extracting ../cifar10_data/cifar-10-python.tar.gz to ../cifar10_data/


##3. Load your model weights

In [4]:
model_save_name = 'checkpoint.pth'
path = F"/content/gdrive/My Drive/Colab Notebooks/{model_save_name}" 
model = CIFAR_Net()
model.load_state_dict(torch.load(path))
model.cuda()
model.eval()

CIFAR_Net(
  (conv1): Conv2d(3, 80, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(80, 160, kernel_size=(2, 2), stride=(1, 1))
  (conv3): Conv2d(160, 320, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(320, 640, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=640, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=10, bias=True)
)

##4. Check model accuracy

In [5]:
def thresholding(prediction):
    """Find label which shows highest prediction value.
    
    Args:
        prediction (torch.tensor) : prediction tensor. Dimension of [Batch size, 10]
    
    Returns:
        pred_label (torch.tensor) : Index which shows maximum prediction value. Dimension of [Batch size]
    """
    _, pred_label = torch.max(prediction, 1)
    return pred_label

In [6]:
model.eval()
total_cnt, correct_cnt = 0.0, 0.0
for batch_idx, (x, target) in enumerate(test_loader):
    with torch.no_grad():
      x, target = x.cuda(), target.cuda()
      prediction = model(x)
      total_cnt += x.data.size(0)
      correct_cnt += (thresholding(prediction) == target.data).sum().item()

print(f"Test Accuracy = {100*correct_cnt/total_cnt}%")

Test Accuracy = 74.88%
